In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import keras
from keras import backend as K
from keras.preprocessing import image
from keras.models import Model

import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import Sequential
from keras.layers import Dense, Activation, Flatten

import numpy as np
np.random.seed(123)
import gzip
import _pickle as cPickle
import os
from collections import Counter

from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.neighbors import KDTree

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
def get_mod():
    vgg_mod = tf.keras.applications.VGG16(include_top=True, weights='imagenet')
    # vgg_mod = keras.applications.VGG16(include_top=True, weights='imagenet', input_shape = [128,128,3])
    # vgg_mod.layers.pop()
    # vgg_mod.layers.pop()

    inp = vgg_mod.input
    out = vgg_mod.layers[-3].output

    # preds = Dense(4096, activation='softmax')(out)
    model = Model(inp, out)

    return model

def get_feat(model, cropped_image):
    y = image.img_to_array(cropped_image)
    y = np.expand_dims(y, axis=0)
    y = keras.applications.vgg16.preprocess_input(y)
    feat = model.predict(y)
    return feat

In [ ]:
vgg_mod = get_mod()

In [ ]:
from PIL import Image
import glob
from numpy import array as narray
import pickle

loc = ['hamburger', 'caesar_salad', 'caprese_salad', 'carrot_cake', 'chicken', 'chocolate_cake', 'cup_cakes',
             'donuts', 'greek_salad', 'naan', 'paneer', 'roti', 'vegetable']
names = ['Burger', 'Caesar Salad', 'Caprese Salad', 'Carrot Cake', 'Butter Chicken', 'Chocolate Cake', 'Cup Cake',
         'Donut', 'Greek Salad', 'Naan', 'Paneer Butter Masala', 'Roti', 'Mixed Vegetable Curry']
for k in range(len(loc)):
    fname = glob.glob(f"/content/gdrive/MyDrive/400 pic Dataset/{loc[k]}/*jpg")
    fname.sort()

    l = []
    a = []
    c = 0
    for i in fname:
      IMAGEPATH = i
      img = Image.open(IMAGEPATH).resize((224, 224))
      img_feature = get_feat(vgg_mod, img)
      img_feature = normalize(img_feature, norm='l2')

      a.append(img_feature)
      a.insert(c, names[k])
      c+=2

    b = []
    j = 1
    c = []
    for i in range(0, len(a), 2):
      b.append(a[i])
      b.append(a[i+j])
      c.append(b)
      b = []
    if names[k] == 'Burger':
      d = []
    for i in range(0, len(c)):
      d.append(tuple(c[i]))

f_name = "final.pkl"

open_file = open(f_name, "wb")
pickle.dump(d, open_file)
open_file.close()